In [20]:
import requests

# Global constants for token mint addresses
SOL_MINT = "So11111111111111111111111111111111111111112"  # Wrapped SOL mint address
USDC_MINT = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"  # USDC mint address on Solana mainnet

def get_swap_quote():
    # Updated Jupiter API endpoint for swap quotes
    url = "https://api.jup.ag/swap/v1/quote"
    
    # Amount of input SOL in lamports (50,000,000 lamports ~ 0.05 SOL)
    amount = "5000000"
    
    # Your wallet's public key (replace with your actual public key)
    user_public_key = "YOUR_WALLET_PUBLIC_KEY"

    # Set up query parameters
    params = {
        "inputMint": SOL_MINT,
        "outputMint": USDC_MINT,
        "amount": amount,
        "userPublicKey": user_public_key,
        "slippage": 1  # 1% slippage tolerance
    }

    # For free usage you may not need an API key, but for higher rate limits, include it in headers:
    # headers = {"Authorization": "Bearer YOUR_API_KEY"}
    response = requests.get(url, params=params)  # , headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

def print_friendly_quote(quote):
    # Convert input amount from lamports to SOL (1 SOL = 1e9 lamports)
    in_amount = int(quote.get("inAmount", "0"))
    sol_amount = in_amount / 1e9

    # Convert output amount from base units (USDC typically has 6 decimals)
    out_amount = int(quote.get("outAmount", "0"))
    usdc_amount = out_amount / 1e6

    swap_mode = quote.get("swapMode", "N/A")
    slippage_bps = quote.get("slippageBps", "N/A")
    swap_usd_value = quote.get("swapUsdValue", "N/A")
    context_slot = quote.get("contextSlot", "N/A")

    # Extract route information (assumes at least one route is available)
    route_plan = quote.get("routePlan", [])
    if route_plan:
        swap_info = route_plan[0].get("swapInfo", {})
        route_label = swap_info.get("label", "N/A")
        fee_amount_str = swap_info.get("feeAmount", "0")
        fee_mint = swap_info.get("feeMint", "")
        try:
            fee_amount = int(fee_amount_str)
        except Exception:
            fee_amount = 0

        # Determine fee conversion factor based on feeMint
        if fee_mint == USDC_MINT:
            fee_converted = fee_amount / 1e6  # USDC has 6 decimals
            fee_token = "USDC"
        elif fee_mint == SOL_MINT:
            fee_converted = fee_amount / 1e9  # SOL has 9 decimals
            fee_token = "SOL"
        else:
            fee_converted = fee_amount  # Unknown, no conversion
            fee_token = "units"
    else:
        route_label = "N/A"
        fee_converted = "N/A"
        fee_token = ""

    print("\n----- Jupiter Swap Quote Summary -----")
    print(f"Swap Mode: {swap_mode}")
    print(f"Input: {sol_amount:.9f} SOL (from {in_amount} lamports)")
    print(f"Expected Output: {usdc_amount:.6f} USDC (approx.)")
    print(f"Platform Fee: {fee_converted} {fee_token}")
    try:
        # Convert slippage to percentage (if given in basis points)
        slippage_pct = float(slippage_bps) / 100 if slippage_bps != "N/A" else "N/A"
        print(f"Slippage Tolerance: {slippage_bps} bps (i.e., {slippage_pct:.2f}%)")
    except Exception:
        print(f"Slippage Tolerance: {slippage_bps}")
    print(f"Estimated Swap USD Value: ${swap_usd_value}")
    print(f"Route Used: {route_label}")
    print(f"Context Slot: {context_slot}")
    print("----------------------------------------\n")

if __name__ == "__main__":
    quote = get_swap_quote()
    if quote:
        print("Raw swap quote data from Jupiter:")
        print(quote)
        print_friendly_quote(quote)
    else:
        print("Failed to fetch swap quote data.")


Raw swap quote data from Jupiter:
{'inputMint': 'So11111111111111111111111111111111111111112', 'inAmount': '5000000', 'outputMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'outAmount': '866987', 'otherAmountThreshold': '862653', 'swapMode': 'ExactIn', 'slippageBps': 50, 'platformFee': None, 'priceImpactPct': '0.0001580622477600000000000044', 'routePlan': [{'swapInfo': {'ammKey': 'GMgh4NtWrGaUf1RR2kcXD7LY1jou1qFAuSsQeKp5ow4a', 'label': 'Meteora DLMM', 'inputMint': 'So11111111111111111111111111111111111111112', 'outputMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'inAmount': '5000000', 'outAmount': '866987', 'feeAmount': '535', 'feeMint': 'So11111111111111111111111111111111111111112'}, 'percent': 100}], 'scoreReport': None, 'contextSlot': 321680327, 'timeTaken': 0.001298617, 'swapUsdValue': '0.867124059577943665932755986', 'simplerRouteUsed': False}

----- Jupiter Swap Quote Summary -----
Swap Mode: ExactIn
Input: 0.005000000 SOL (from 5000000 lamports)
Expected Output: 

In [1]:
import requests
import json

# Global token mint addresses
SOL_MINT = "So11111111111111111111111111111111111111112"   # Wrapped SOL
USDC_MINT = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"  # USDC

def get_swap_quote():
    url = "https://api.jup.ag/swap/v1/quote"
    
    # Amount of input SOL in lamports (50,000,000 lamports ≈ 0.05 SOL)
    amount = "50000000"
    user_public_key = "ExNHytNiuAVTuWLvt3DcKLCpV2DzqwgS3qEys3gJBYNs"  # Replace with your actual wallet public key

    params = {
        "inputMint": SOL_MINT,
        "outputMint": USDC_MINT,
        "amount": amount,
        "userPublicKey": user_public_key,
        "slippage": 1  # 1% slippage tolerance
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching quote:", response.status_code, response.text)
        return None

def perform_swap_with_best_route(quote):
    url = "https://api.jup.ag/swap/v1/swap"
    user_public_key = "ExNHytNiuAVTuWLvt3DcKLCpV2DzqwgS3qEys3gJBYNs"  # Replace with your actual wallet public key

    # Send the full routePlan array (not just the first element)
    payload = {
        "route": quote["routePlan"],  # Use the entire routePlan array
        "userPublicKey": user_public_key,
        "simulate": False,
        "wrapUnwrapSOL": False
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error performing swap:", response.status_code, response.text)
        return None

    # Build payload with only the best route (a single routePlan leg)
    payload = {
        "route": best_route,
        "userPublicKey": user_public_key,
        "simulate": False,
        "wrapUnwrapSOL": False
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error performing swap:", response.status_code, response.text)
        return None

def print_friendly_quote(quote):
    in_amount = int(quote.get("inAmount", "0"))
    sol_amount = in_amount / 1e9
    out_amount = int(quote.get("outAmount", "0"))
    usdc_amount = out_amount / 1e6

    swap_mode = quote.get("swapMode", "N/A")
    slippage_bps = quote.get("slippageBps", "N/A")
    swap_usd_value = quote.get("swapUsdValue", "N/A")
    context_slot = quote.get("contextSlot", "N/A")

    # Extract fee info from first routePlan leg
    route_plan = quote.get("routePlan", [])
    if route_plan:
        swap_info = route_plan[0].get("swapInfo", {})
        route_label = swap_info.get("label", "N/A")
        fee_amount_str = swap_info.get("feeAmount", "0")
        fee_mint = swap_info.get("feeMint", "")
        try:
            fee_amount = int(fee_amount_str)
        except Exception:
            fee_amount = 0

        if fee_mint == USDC_MINT:
            fee_converted = fee_amount / 1e6
            fee_token = "USDC"
        elif fee_mint == SOL_MINT:
            fee_converted = fee_amount / 1e9
            fee_token = "SOL"
        else:
            fee_converted = fee_amount
            fee_token = "units"
    else:
        route_label = "N/A"
        fee_converted = "N/A"
        fee_token = ""
    
    print("\n----- Jupiter Swap Quote Summary -----")
    print(f"Swap Mode: {swap_mode}")
    print(f"Input: {sol_amount:.9f} SOL (from {in_amount} lamports)")
    print(f"Expected Output: {usdc_amount:.6f} USDC (approx.)")
    print(f"Platform Fee: {fee_converted} {fee_token}")
    try:
        slippage_pct = float(slippage_bps) / 100 if slippage_bps != "N/A" else "N/A"
        print(f"Slippage Tolerance: {slippage_bps} bps (i.e., {slippage_pct:.2f}%)")
    except Exception:
        print(f"Slippage Tolerance: {slippage_bps}")
    print(f"Estimated Swap USD Value: ${swap_usd_value}")
    print(f"Route Used: {route_label}")
    print(f"Context Slot: {context_slot}")
    print("----------------------------------------\n")

if __name__ == "__main__":
    quote = get_swap_quote()
    if quote:
        print("Raw swap quote data from Jupiter:")
        print(json.dumps(quote, indent=2))
        print_friendly_quote(quote)
        
        swap_tx = perform_swap_with_best_route(quote)
        if swap_tx:
            print("\n----- Jupiter Swap Transaction Data -----")
            print(json.dumps(swap_tx, indent=2))
            print("-------------------------------------------")
            print("NOTE: You must sign this transaction with your wallet's private key and submit it to the Solana network.")
        else:
            print("Failed to perform swap.")
    else:
        print("Failed to fetch swap quote data.")


Raw swap quote data from Jupiter:
{
  "inputMint": "So11111111111111111111111111111111111111112",
  "inAmount": "50000000",
  "outputMint": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
  "outAmount": "8688340",
  "otherAmountThreshold": "8644899",
  "swapMode": "ExactIn",
  "slippageBps": 50,
  "platformFee": null,
  "priceImpactPct": "0",
  "routePlan": [
    {
      "swapInfo": {
        "ammKey": "4WNy4aYuAo98y6k7tVizzT9gYvAmE9YKK25jaYsq6f8t",
        "label": "ZeroFi",
        "inputMint": "So11111111111111111111111111111111111111112",
        "outputMint": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
        "inAmount": "50000000",
        "outAmount": "8688340",
        "feeAmount": "0",
        "feeMint": "11111111111111111111111111111111"
      },
      "percent": 100
    }
  ],
  "scoreReport": null,
  "contextSlot": 321682103,
  "timeTaken": 0.001049126,
  "swapUsdValue": "8.684870398197660111795473108",
  "simplerRouteUsed": false
}

----- Jupiter Swap Quote Summary --